## Penjelasan

Langkah pertama yang dilakukan adalah meng-import modul pandas dan numpy serta mengambil data lapor.csv pada folder di data_path

In [18]:
import pandas as pd
import numpy as np

In [15]:
data_pathh = "data/data_source/"
filename = "source1.csv"

In [22]:
train_df2 = pd.read_csv(data_pathh + filename, delimiter='')
train_df2.head()

ValueError: only single character unicode strings can be converted to Py_UCS4, got length 0

In [23]:
data_path = "data/data_source/"
filename = "source1.csv"

Membaca file lapor.csv dengan memanfaatkan modul Pandas dengan menggunakan fungsi .head(), maka data yang ditampilkan merupakan 5 data teratas 

In [25]:
train_df = pd.read_csv(data_path + filename, delimiter=';')
train_df.head()

UnicodeDecodeError: 'utf-8' codec can't decode byte 0xa0 in position 239: invalid start byte

menampilkan jumlah masing-masing kategori

In [4]:
kategori_total_list = train_df["Kategori"].value_counts()
with pd.option_context('display.max_rows', None, 'display.max_columns', None):
    print (kategori_total_list)

HUKUM MEMBACA DO?A DI TENGAH SHOLAT                                        7
Hari terjadinya isra' dan mi'raj                                           7
ALASAN BERSUCI DENGAN AIR                                                  7
HUKUM MELAKUKAN HUBUNGAN INTIM PADA MASA MENYUSUI                          7
HUKUM SEMBELIHAN NON MUSLIM                                                7
Hukum makan sampai kekenyangan                                             7
HUKUM LELAKI MEMAKAI KALUNG TASBIH                                         7
apakah makan sahur mencukupi sebagai niat puasa                            7
hal-hal yang makruh dilakukan saat shalat                                  7
hukum sholat qobliyah shubuh setelah shalat shubuh                         7
BOLEHKAH SUAMI MENGELUARKAN MANI SAAT BERCUMBU DENGAN ISTRI YANG HAID ?    7
munakahat: hukum menikahi ibu dari istri yang tidak sah akad nikahnya      7
status hukum kesucian air dalam ember yang dialiri air keran               7

menggabungkan kategori yang memiliki jumlah berita sedikit dengan list kategori lain yang lebih umum dan memiliki jumlah yang sudah banyak.

In [5]:
kategori = train_df["Kategori"].values

Menampilkan isi pertanyaan

In [6]:
pertanyaan = train_df["pertanyaan"].values
pertanyaan

array(['bagaimana kisah tentang mukasyafah nya sayyidina ali r.a tentang sayyidina umar vs mungkar nakir',
       'ceritakan kisah sayyidina umar vs munkar nakir',
       'kisah sayyidina umar vs munkar nakir',
       'sayyidina umar vs malaikat munkar nakir',
       'sayyidina umar vs munkar nakir',
       'cerita sayyidina umar vs munkar nakir termaktub dari kitab apa?',
       'kenapa wudhu menggunakan air?',
       'apakah bisa wudhu tidak menggunakan air?',
       'kalau tidak ada air bagaimana bisa berwuduhu?', 'wudhu tanpa air',
       'mengapa wuduh harus menggunakan air?', 'haruskah wudhu pake air?',
       "kenapa sholat maghrib 3 rokaat. Ashar 4 Zhuhur 4 Subuh 2 roka'at. Kok tidak disamakan saja",
       'bolehkah mahar nikah berasal dari orang lain?',
       'calon suami mendapat sponsor untuk dijadikan mahar nikah seperti pada acara tv',
       'apakah mahar harus dari calon suami saja?',
       'bagaimana jika mahar nikah berasal dari orang lain?',
       'calon suami mem

Mendapatkan word embedding dari database glove

In [7]:
def getWordEmbedding(word, cursor):
#     word = word.replace("'", "''")
    sql = """select vec from term where term like %s"""
    cursor.execute(sql, (str(word),))
    data = cursor.fetchall()
    if len(data) > 0:
        decoded_vec = json.JSONDecoder().decode(data[0][0])
        vec = np.asarray(decoded_vec, dtype=np.float32)
        return True, vec
    else:
        return False, data
    
def myTokenizer(content, lower=True):
    raw = content.split(' ')
    remover = re.compile("[^a-zA-Z-]")
    
    token = []
    
    for i in raw:
        term = remover.sub('', i)
        if lower == True:
            term = term.lower()
        token.append(term)
#     tokenized = filter(None, token)
    
#     return tokenized
    return token

def sentenceToVec(string):

    if type(string) is not str:
        return np.zeros((300,), dtype=float)
   
    string = string.replace('\n', '')
    string = np.array(myTokenizer(string))
#     print(string)
    feature = None
    begin = True
    for word in string:
        stat, vec = getWordEmbedding(word, cursor)
        if not stat:
            continue
        if begin:
            begin = False
            feature = vec
        else:
            feature += vec
#             feature = np.concatenate([feature, vec])
#     print('running') 
    if feature is not None:
        feature = feature/np.linalg.norm(feature)
    else:
        feature = np.zeros((300,), dtype=float)
    
    return feature

import re
import json
import unicodedata
import mysql.connector
from tqdm import tqdm

db = mysql.connector.connect(user="root", password='', database="glove")
cursor = db.cursor(buffered=True)

pbar = tqdm(total=len(pertanyaan))
ftr = []
for i in pertanyaan:
    try:
        ftr.append(sentenceToVec(i))
    except Exception as err:
        print (err)
        print (i)
        break
    pbar.update(1)
pbar.close()
ftr = np.array(ftr)

100%|███████████████████████████████████████████| 445/445 [00:15<00:00, 27.95it/s]


In [8]:
nan_idx = []
for i in range(len(kategori)):
    if kategori[i] is np.nan:
        nan_idx.append(i)
kategori_c = np.delete(kategori, nan_idx)
ftr_c = np.delete(ftr, nan_idx, 0)

In [9]:
cls = []
kategori_u = pd.unique(kategori_c)
kategori_u = kategori_u.tolist()
for i in range(len(kategori_c)):
    one_hot = np.zeros((len(kategori_u),), dtype=int)
    idx = kategori_u.index(kategori_c[i])
    one_hot[idx] = 1
    cls.append(one_hot)
cls = np.array(cls)

print ("Mengecek array size dari fitur dan kelas...")
print ("size of ftr " + str(ftr_c.shape))
print ("size of cls " + str(cls.shape))

Mengecek array size dari fitur dan kelas...
size of ftr (445, 300)
size of cls (445, 72)


In [10]:
np.save(data_path + "ftr-data-target.npy", ftr_c)
np.save(data_path + "cls-data-target.npy", cls)

## Splitting Dataset
Memisahkan Dataset menjadi Train Set dan Test Tes

In [11]:
ftr_data_target = np.load(data_path + "ftr-data-target.npy")
cls_data_target = np.load(data_path + "cls-data-target.npy")

In [12]:
data_target_path_splitted = "data/splitted/data_target/"

In [13]:
from sklearn.model_selection import train_test_split, StratifiedShuffleSplit

sss = StratifiedShuffleSplit(n_splits=3, test_size=0.4, random_state=0)
for train_index, test_index in sss.split(ftr_data_target, cls_data_target):
    X, X_test = ftr_data_target[train_index], ftr_data_target[test_index]
    Y, Y_test = cls_data_target[train_index], cls_data_target[test_index]
    break
# X = np.expand_dims(X, axis=2)
# X_test = np.expand_dims(X_test, axis=2)

In [14]:
np.save(data_target_path_splitted + "X.npy", X)
np.save(data_target_path_splitted + "Y.npy", Y)
np.save(data_target_path_splitted + "X_test.npy", X_test)
np.save(data_target_path_splitted + "Y_test.npy", Y_test)